In [1]:
!pip install requests pandas

In [3]:
import requests
from datetime import datetime
import pandas as pd
from time import sleep

class MemecoinScraper:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://pro-api.coinmarketcap.com/v1'
        self.headers = {
            'X-CMC_PRO_API_KEY': api_key,
            'Accept': 'application/json'
        }

    def get_all_cryptocurrencies(self, start=1, limit=5000):
        """Fetch all cryptocurrencies with their launch dates"""
        endpoint = f"{self.base_url}/cryptocurrency/listings/latest"

        params = {
            'start': start,
            'limit': limit,
            'convert': 'USD',
            'aux': 'date_added,tags'
        }

        try:
            response = requests.get(endpoint, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error fetching cryptocurrencies: {e}")
            return None

    def filter_memecoins(self, cryptocurrencies, start_date='2022-10-01'):
        """Filter memecoins created after the specified date"""
        start_timestamp = datetime.strptime(start_date, '%Y-%m-%d')

        memecoins = []
        for crypto in cryptocurrencies:
            # Check if 'meme' is in tags and creation date is after October 2022
            date_added = datetime.strptime(crypto['date_added'][:10], '%Y-%m-%d')
            if ('memes' in crypto.get('tags', []) or 'meme' in crypto.get('tags', [])) and date_added > start_timestamp:
                memecoins.append({
                    'name': crypto['name'],
                    'symbol': crypto['symbol'],
                    'date_added': crypto['date_added'],
                    'price_usd': crypto['quote']['USD']['price'],
                    'market_cap': crypto['quote']['USD']['market_cap'],
                    'volume_24h': crypto['quote']['USD']['volume_24h'],
                    'percent_change_24h': crypto['quote']['USD']['percent_change_24h']
                })

        return memecoins

    def save_to_csv(self, memecoins, filename='memecoins_data.csv'):
        """Save memecoin data to CSV file"""
        df = pd.DataFrame(memecoins)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

def main():
    # Initialize with your API key
    api_key = 'ENTER API KEY HERE    scraper = MemecoinScraper(api_key)

    # Get all cryptocurrencies
    print("Fetching cryptocurrency data...")
    cryptos = scraper.get_all_cryptocurrencies()

    if cryptos:
        # Filter memecoins
        print("Filtering memecoins...")
        memecoins = scraper.filter_memecoins(cryptos)

        # Save results
        print(f"Found {len(memecoins)} memecoins created after October 2022")
        scraper.save_to_csv(memecoins)
    else:
        print("Failed to fetch cryptocurrency data")

if __name__ == "__main__":
    main()

Fetching cryptocurrency data...
Filtering memecoins...
Found 831 memecoins created after October 2022
Data saved to memecoins_data.csv


In [4]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from time import sleep

class MemecoinScraper:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://pro-api.coinmarketcap.com/v1'
        self.headers = {
            'X-CMC_PRO_API_KEY': api_key,
            'Accept': 'application/json'
        }

    def get_all_cryptocurrencies(self, start=1, limit=5000):
        """Fetch all cryptocurrencies with their launch dates"""
        endpoint = f"{self.base_url}/cryptocurrency/listings/latest"

        params = {
            'start': start,
            'limit': limit,
            'convert': 'USD',
            'aux': 'date_added,tags'
        }

        try:
            response = requests.get(endpoint, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error fetching cryptocurrencies: {e}")
            return None

    def filter_memecoins(self, cryptocurrencies, start_date='2022-10-01'):
        """Filter memecoins created after start_date and at least 1 year old"""
        start_timestamp = datetime.strptime(start_date, '%Y-%m-%d')
        current_time = datetime.now()
        one_year_ago = current_time - timedelta(days=365)

        memecoins = []
        for crypto in cryptocurrencies:
            date_added = datetime.strptime(crypto['date_added'][:10], '%Y-%m-%d')

            # Check if:
            # 1. It's a memecoin (has meme tag)
            # 2. Created after October 2022
            # 3. Has been around for at least a year
            if (('memes' in crypto.get('tags', []) or 'meme' in crypto.get('tags', [])) and
                date_added > start_timestamp and
                date_added <= one_year_ago):

                days_since_creation = (current_time - date_added).days

                memecoins.append({
                    'name': crypto['name'],
                    'symbol': crypto['symbol'],
                    'date_added': crypto['date_added'],
                    'days_since_creation': days_since_creation,
                    'price_usd': crypto['quote']['USD']['price'],
                    'market_cap': crypto['quote']['USD']['market_cap'],
                    'volume_24h': crypto['quote']['USD']['volume_24h'],
                    'percent_change_24h': crypto['quote']['USD']['percent_change_24h']
                })

        # Sort by market cap
        memecoins.sort(key=lambda x: x['market_cap'], reverse=True)
        return memecoins

    def save_to_csv(self, memecoins, filename='memecoins_data.csv'):
        """Save memecoin data to CSV file"""
        df = pd.DataFrame(memecoins)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

        # Print summary statistics
        if not df.empty:
            print("\nSummary Statistics:")
            print(f"Total number of qualifying memecoins: {len(df)}")
            print(f"Average age in days: {df['days_since_creation'].mean():.0f}")
            print(f"Total market cap: ${df['market_cap'].sum():,.2f}")
            print("\nTop 5 by market cap:")
            print(df[['name', 'market_cap', 'days_since_creation']].head().to_string())

def main():
    # Initialize with your API key
    api_key = 'c9bf995e-4819-43b0-bfc4-396782d7ecb4'
    scraper = MemecoinScraper(api_key)

    # Get all cryptocurrencies
    print("Fetching cryptocurrency data...")
    cryptos = scraper.get_all_cryptocurrencies()

    if cryptos:
        # Filter memecoins
        print("Filtering memecoins...")
        memecoins = scraper.filter_memecoins(cryptos)

        # Save results
        print(f"Found {len(memecoins)} memecoins that meet all criteria:")
        print("- Created after October 2022")
        print("- At least 1 year old")
        print("- Tagged as meme/memes")

        scraper.save_to_csv(memecoins)
    else:
        print("Failed to fetch cryptocurrency data")

if __name__ == "__main__":
    main()

Fetching cryptocurrency data...
Filtering memecoins...
Found 119 memecoins that meet all criteria:
- Created after October 2022
- At least 1 year old
- Tagged as meme/memes
Data saved to memecoins_data.csv

Summary Statistics:
Total number of qualifying memecoins: 119
Average age in days: 528
Total market cap: $15,149,946,652.62

Top 5 by market cap:
           name    market_cap  days_since_creation
0          Pepe  6.494706e+09                  613
1          Bonk  2.000511e+09                  721
2     dogwifhat  1.824960e+09                  367
3      Mog Coin  8.155384e+08                  518
4  Popcat (SOL)  6.805233e+08                  366


In [5]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from time import sleep

class MemecoinScraper:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://pro-api.coinmarketcap.com/v1'
        self.headers = {
            'X-CMC_PRO_API_KEY': api_key,
            'Accept': 'application/json'
        }

    def get_historical_data(self, coin_id, time_period='365d'):
        """Fetch historical price data for a specific coin"""
        endpoint = f"{self.base_url}/cryptocurrency/quotes/historical"

        # Calculate time range
        end_time = datetime.now()
        start_time = end_time - timedelta(days=365)

        params = {
            'id': coin_id,
            'time_start': start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ'),
            'time_end': end_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ'),
            'interval': '1d',  # Daily intervals
            'convert': 'USD'
        }

        try:
            response = requests.get(endpoint, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error fetching historical data for coin {coin_id}: {e}")
            return None

    def analyze_price_pattern(self, historical_data):
        """
        Analyze price patterns to identify declining trends and pump-and-dump patterns
        Returns: dict with analysis results
        """
        prices = [point['quote']['USD']['price'] for point in historical_data]

        # Calculate metrics
        max_price = max(prices)
        current_price = prices[-1]
        price_decline = ((max_price - current_price) / max_price) * 100

        # Detect peaks and crashes
        peaks = []
        crashes = []
        window_size = 7  # 7-day window for peak/crash detection

        for i in range(window_size, len(prices) - window_size):
            window = prices[i-window_size:i+window_size]
            if prices[i] == max(window):
                peaks.append((i, prices[i]))

            # Detect crashes (price drops of more than 20% in 3 days)
            if i >= 3:
                three_day_decline = ((prices[i-3] - prices[i]) / prices[i-3]) * 100
                if three_day_decline > 20:
                    crashes.append((i, prices[i], three_day_decline))

        # Calculate trend
        is_declining = current_price < np.mean(prices[:30])  # Compare to 30-day average
        has_pump_and_dump = len(peaks) > 0 and len(crashes) > 0

        return {
            'price_decline': price_decline,
            'peak_count': len(peaks),
            'crash_count': len(crashes),
            'is_declining': is_declining,
            'has_pump_and_dump': has_pump_and_dump,
            'max_price': max_price,
            'current_price': current_price
        }

    def get_all_cryptocurrencies(self, start=1, limit=5000):
        """Fetch all cryptocurrencies with their launch dates"""
        endpoint = f"{self.base_url}/cryptocurrency/listings/latest"

        params = {
            'start': start,
            'limit': limit,
            'convert': 'USD',
            'aux': 'date_added,tags,platform,id'
        }

        try:
            response = requests.get(endpoint, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error fetching cryptocurrencies: {e}")
            return None

    def filter_memecoins(self, cryptocurrencies, start_date='2022-10-01'):
        """Filter memecoins based on all criteria"""
        start_timestamp = datetime.strptime(start_date, '%Y-%m-%d')
        current_time = datetime.now()
        one_year_ago = current_time - timedelta(days=365)

        memecoins = []
        for crypto in cryptocurrencies:
            date_added = datetime.strptime(crypto['date_added'][:10], '%Y-%m-%d')

            # Initial filtering
            if (('memes' in crypto.get('tags', []) or 'meme' in crypto.get('tags', [])) and
                date_added > start_timestamp and
                date_added <= one_year_ago):

                print(f"Analyzing price patterns for {crypto['name']}...")
                historical_data = self.get_historical_data(crypto['id'])
                sleep(1)  # Respect API rate limits

                if historical_data:
                    price_analysis = self.analyze_price_pattern(historical_data)

                    # Only include if it shows declining pattern or pump-and-dump behavior
                    if price_analysis['is_declining'] or price_analysis['has_pump_and_dump']:
                        memecoins.append({
                            'name': crypto['name'],
                            'symbol': crypto['symbol'],
                            'date_added': crypto['date_added'],
                            'price_usd': crypto['quote']['USD']['price'],
                            'market_cap': crypto['quote']['USD']['market_cap'],
                            'volume_24h': crypto['quote']['USD']['volume_24h'],
                            'price_decline_percent': price_analysis['price_decline'],
                            'peak_count': price_analysis['peak_count'],
                            'crash_count': price_analysis['crash_count'],
                            'max_historical_price': price_analysis['max_price'],
                            'pattern_type': 'Pump & Dump' if price_analysis['has_pump_and_dump'] else 'Declining'
                        })

        # Sort by price decline percentage
        memecoins.sort(key=lambda x: x['price_decline_percent'], reverse=True)
        return memecoins

    def save_to_csv(self, memecoins, filename='declining_memecoins.csv'):
        """Save memecoin data to CSV file with enhanced analysis"""
        df = pd.DataFrame(memecoins)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

        if not df.empty:
            print("\nSummary Statistics:")
            print(f"Total number of qualifying memecoins: {len(df)}")
            print(f"Average price decline: {df['price_decline_percent'].mean():.2f}%")
            print(f"Coins with pump & dump patterns: {len(df[df['pattern_type'] == 'Pump & Dump'])}")
            print(f"Coins with declining patterns: {len(df[df['pattern_type'] == 'Declining'])}")
            print("\nTop 5 largest price declines:")
            print(df[['name', 'price_decline_percent', 'pattern_type']].head().to_string())

def main():
    api_key = 'c9bf995e-4819-43b0-bfc4-396782d7ecb4'
    scraper = MemecoinScraper(api_key)

    print("Fetching cryptocurrency data...")
    cryptos = scraper.get_all_cryptocurrencies()

    if cryptos:
        print("Analyzing price patterns...")
        memecoins = scraper.filter_memecoins(cryptos)

        print(f"\nFound {len(memecoins)} memecoins that meet all criteria:")
        print("- Created after October 2022")
        print("- At least 1 year old")
        print("- Tagged as meme/memes")
        print("- Shows declining trend or pump-and-dump pattern")

        scraper.save_to_csv(memecoins)
    else:
        print("Failed to fetch cryptocurrency data")

if __name__ == "__main__":
    main()

Fetching cryptocurrency data...
Error fetching cryptocurrencies: 400 Client Error: Bad Request for url: https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?start=1&limit=5000&convert=USD&aux=date_added%2Ctags%2Cplatform%2Cid
Failed to fetch cryptocurrency data
